In [1]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
def get_stock_map_load():
    data = pd.read_csv('C:/big18/final/with/almost/전종목시세.csv', encoding='cp949')
    data =data[['종목코드','종목명','상장주식수']]
    result_map = {row['종목코드']: {'종목명': row['종목명'], '상장주식수': row['상장주식수']} for index, row in data.iterrows()}
    return result_map

def get_stock_share(code):
    map = get_stock_map_load()
    stock_share = map[f'{code}']['상장주식수']
    return stock_share

def get_stock_name(code):
    map = get_stock_map_load()
    stock_name = map[f'{code}']['종목명']
    return stock_name

In [7]:
code = '000660' # 전처리할 종목 코드

In [30]:
# 상장주식수로 거래량 스케일링
def divide_volumes(row,share):
    try:
        divisor = share
        row['power'] = row['매수량'] - row['매도량']
        row['power'] = row['power'] / divisor
    except KeyError:
        print(f"코드 {row['code']}를 찾지 못했습니다.")
        input()
    return row

In [31]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta

# 결과를 저장할 리스트


stock_name = get_stock_name(code) # 해당 코드의 종목명저장
stock_share = get_stock_share(code) # 해당 코드의 상장주식 수

df = pd.read_csv(f'C:\\big18\\final\\with\\almost\\real_data\\{code}.csv',encoding='cp949')
df = df[['code','종가','매수량','매도량']]
df = divide_volumes(df, stock_share)


In [32]:
df

,code,종가,매수량,매도량,power
0,660,96800,87281.0,11919.0,0.000104
1,660,97000,39787.0,15861.0,0.000033
2,660,96200,19856.0,17740.0,0.000003
3,660,96300,12955.0,15239.0,-0.000003
4,660,96400,11016.0,8216.0,0.000004
...,...,...,...,...,...
186835,660,193200,4017.0,7343.0,-0.000005
186836,660,193000,5232.0,26959.0,-0.000030
186837,660,192800,2840.0,6745.0,-0.000005
186838,660,193000,12271.0,4646.0,0.000010


In [33]:

# 데이터 로드
df=df[['code','종가', 'power']]
df.rename(columns={'종가' : 'close'}, inplace=True)


C:\Users\G013\AppData\Local\Temp\ipykernel_5176\1101726380.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'종가' : 'close'}, inplace=True)


In [40]:
new_df = pd.DataFrame()
n = len(df)
close_values = []
power_values = []
target_values = []
for i in range(n - 4):
    close_value = (df['close'].iloc[i + 1] - df['close'].iloc[i]) / df['close'].iloc[i] * 10000
    power_value = (df['power'].iloc[i + 1] - df['power'].iloc[i]) * 1000000
    if df['close'].iloc[i + 4] - df['close'].iloc[i + 1] > 0:
        target_value = 2
    elif df['close'].iloc[i + 4] - df['close'].iloc[i + 1] == 0:
        target_value = 1
    else:
        target_value = 0
    close_values.append(round(close_value, 4))
    power_values.append(round(power_value, 4))
    target_values.append(target_value)
new_df['close'] = close_values
new_df['power'] = power_values
new_df['target'] = target_values

In [41]:
new_df.describe()

,close,power,target
count,186836.000000,186836.000000,186836.000000
mean,0.047128,-0.000714,0.990692
std,14.293235,21.094624,0.833424
min,-567.307700,-1042.052900,0.000000
25%,-7.315300,-3.787100,0.000000
50%,0.000000,-0.004100,1.000000
75%,7.199400,3.756900,2.000000
max,625.698300,1285.918900,2.000000


In [42]:
new_df


,close,power,target
0,20.6612,-70.6536,0
1,-82.4742,-29.9587,2
2,10.3950,-6.0439,0
3,10.3842,6.9835,0
4,-10.3734,-8.0068,0
...,...,...,...
186831,5.1787,2.5151,1
186832,-5.1760,-1.2280,0
186833,0.0000,2.8187,0
186834,5.1787,-2.9135,0


In [43]:
new_df.to_csv('./new_data/000660.csv', index=False)

In [ ]:

# Unique 코드 및 카운트 계산
counts = df['code'].value_counts()
counts_dict = counts.to_dict()

# 결과를 저장할 리스트
results = []

for key in tqdm(counts_dict.keys()):
    value = counts_dict[key]
    for i in range(value - 12):   # 예측 분 -1
        start_idx = counts_dict[key] - value + i
        end_idx = start_idx + 9  # 묶을 분 -1

        # 벡터화하여 연산
        vectorList = []
        # for col in ['close','power']:
        #     diff = (df[col].values[start_idx:end_idx] - df[col].values[start_idx + 1:end_idx + 1]) 
        #     vectorList.extend(round(d, 2) for d in diff)
        diff = (df['close'].values[start_idx:end_idx] - df['close'].values[start_idx + 1:end_idx + 1]) / df['close'].values[start_idx:end_idx] * 10000
        vectorList.extend(round(d, 4) for d in diff)
        diff = (df['power'].values[start_idx:end_idx] - df['power'].values[start_idx + 1:end_idx + 1])
        vectorList.extend(round(d, 2) for d in diff)

        openValue = df['close'][start_idx + 12] - df['close'][start_idx + 9]
        if openValue > 0:
            vectorList.append(2.0)
        elif openValue == 0:
            vectorList.append(1.0)
        else:
            vectorList.append(0.0)

        results.append(vectorList)
        
    print(f'{code} {stock_name} 처리 완료')

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# CSV 파일에 저장
results_df.to_csv(f'C:\\big18\\final\\with\\almost\\data\\{code}전체데이터Vector.csv', index=False)

In [6]:
# import pandas as pd
# import glob

# # CSV 파일들이 있는 디렉토리 경로
# path = './trainStockData/'  # 여기에 CSV 파일들이 있는 경로를 입력하세요
# all_files = glob.glob(path + "/*.csv")  # 모든 CSV 파일 목록 가져오기
# all_files
# # 모든 CSV 파일을 읽고 하나의 데이터프레임으로 합치기
# dataframes = []
# for filename in all_files:
#     df = pd.read_csv(filename)  # CSV 파일 읽기
#     dataframes.append(df)  # 데이터프레임 리스트에 추가

# # 모든 데이터프레임을 하나로 합치기
# combined_df = pd.concat(dataframes, ignore_index=True)

# # 결과를 새로운 CSV 파일로 저장
# combined_df.to_csv('combined_data.csv', index=False)  # 원하는 파일명으로 저장
# print('저장완료')
